In [1]:
from owlready2 import *
import json

onto = get_ontology("ontology.owl").load()

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
f = open('../scraping/results/22_23.json')
website = "https://www.zerozero.pt"
file = json.load(f)
print(file['FC Porto'][15])

{'Nome': ['Futebol Clube do Porto'], 'Alcunhas': ['Dragões, Azuis e Brancos, Portistas'], 'Associação': ['AF Porto'], 'Presidente': ['Jorge Nuno de Lima Pinto da Costa'], 'Evolução histórica': ['FC Porto'], 'Morada': ['Estádio do Dragão, Entrada Nascente, Porta 15, Piso 3 Via Futebol Clube do Porto, 4350-415 Porto'], 'Site Oficial': ['http://www.fcporto.pt'], 'E-mail': ['fcporto@fcporto.pt'], 'Rankings': ['16', '60'], 'Hino Oficial': [''], 'Ano de Fundação': ['1893-09-28'], 'Cidade': ['Porto'], 'País': ['Portugal'], 'Marca Equipamento': ['New Balance'], 'Patrocínio': ['Betano | New Balance | Revigrés | Super Bock | MEO'], 'Equipamento': [''], 'Outras Ligações': ['efgi'], 'Num.FPF': ['529']}


In [3]:
league = onto.League("Liga_Portugal", leagueName=["Liga Portugal"])

In [4]:
edition = onto.Edition("22_23", editionYear=["22_23"])
edition.editionOfLeague = league

for x in file:
    clube = onto.Club(file[x][15].get("Nome")[0].replace(" ", "_"), clubFullName=file[x][15].get("Nome"),
                      clubSponsor=file[x][15].get("Patrocínio"), clubZeroZeroLink=[website + file[x][12]],
                      clubWebsite=file[x][15].get("Site Oficial"), clubAdress=file[x][15].get("Morada"),
                      clubPresident=file[x][15].get("Presidente"),
                      clubUefaRanking=[str(file[x][15].get("Rankings"))],
                      clubFoundationYear=file[x][15].get("Ano de Fundação"), clubCity=file[x][15].get("Cidade"),
                      clubEmail=file[x][15].get("E-mail"), clubImage=[file[x][-2]],
                      clubNickName=[file[x][15].get("Alcunhas")],
                      clubCountry=file[x][15].get("País"), clubMainSponsor=file[x][15].get("Marca Equipamento"),
                      clubAssociation=file[x][15].get("Associação"))

    squad = onto.Squad((file[x][15].get("Nome")[0] + "22_23").replace(" ", "_"), squadYear=[23])
    squad.squadParticipatedIn = edition
    clube.clubHasSquad = [squad]
    competitionsWon = file["Benfica"][16]
    for c in competitionsWon:
        item = competitionsWon[c]
        competition = onto.Competition(c.replace(" ", "_"), competitionName=[c], competitionNumber=[item])
        clube.clubWonCompetition.append(competition)

    player_info = file[x][13]
    for player in player_info:
        y = player_info[player]
        player = onto.Player(y[0].replace(" ", "_"), playerimage=[y[11]], playername=[y[0]], playernationality=[y[4]],
                             playernaturalFrom=[y[6]],
                             playerposition=[y[7]], playerweight=[y[10]], playerheight=[y[9]], playerfoot=[y[8]],
                             playerbirthdate=[y[2]], playerage=[y[1]], playerbirthplace=[y[6]])
        player.playerPlayedForSquad = [squad]
        player_records = y[12]
        for name in player_records:
            record = onto.Record(recordcompetitionname=[name], recordgames=[player_records[name][0]],
                                 recordminutes=[player_records[name][1]], recordgoals=[player_records[name][2]],
                                 recordassists=[player_records[name][3]])
            player.playerHasRecord = [record]
            record.recordForSquad = squad

        player_titles = y[13]
        for competition_name in player_titles:
            times = player_titles[competition_name]
            # print(competition_name)
            # print(player_item)
            player_competition = onto.Competition(competition_name.replace(" ", "_"),
                                                  competitionName=[competition_name])
            for i in range(int(times)):
                player.playerWonCompetition.append(player_competition)



In [5]:
editions = ["21_22", "20_21"]

for ed in editions:
    f = open("../scraping/results/" + ed + ".json")
    file = json.load(f)
    edition = onto.Edition(ed, editionYear=[ed])
    edition.editionOfLeague = league

    for x in file:
        if onto[(file[x][15].get("Nome")[0]).replace(" ", "_")] == None:
            clube = onto.Club((file[x][15].get("Nome")[0]).replace(" ", "_"), clubFullName=file[x][15].get("Nome"),
                              clubSponsor=file[x][15].get("Patrocínio"), clubZeroZeroLink=[website + file[x][12]],
                              clubWebsite=file[x][15].get("Site Oficial"), clubAdress=file[x][15].get("Morada"),
                              clubPresident=file[x][15].get("Presidente"),
                              clubUefaRanking=[str(file[x][15].get("Rankings"))],
                              clubFoundationYear=file[x][15].get("Ano de Fundação"), clubCity=file[x][15].get("Cidade"),
                              clubEmail=file[x][15].get("E-mail"), clubImage=[file[x][-2]],
                              clubNickName=[file[x][15].get("Alcunhas")],
                              clubCountry=file[x][15].get("País"), clubMainSponsor=file[x][15].get("Marca Equipamento"),
                              clubAssociation=file[x][15].get("Associação"))

        else:
            clube = onto[(file[x][15].get("Nome")[0]).replace(" ", "_")]

        squad = onto.Squad((file[x][15].get("Nome")[0] + ed).replace(" ", "_"), squadYear=[23])
        squad.squadParticipatedIn = edition
        clube.clubHasSquad.append(squad)

        player_info = file[x][13]
        for player in player_info:

            y = player_info[player]
            if onto[y[0].replace(" ", "_")] == None:
                player = onto.Player(y[0].replace(" ", "_"), playerimage=[y[11]], playername=[y[0]],
                                     playernationality=[y[4]], playernaturalFrom=[y[6]],
                                     playerposition=[y[7]], playerweight=[y[10]], playerheight=[y[9]],
                                     playerfoot=[y[8]], playerbirthdate=[y[2]], playerage=[y[1]],
                                     playerbirthplace=[y[6]])
            else:
                player = onto[y[0].replace(" ", "_")]

            player.playerPlayedForSquad.append(squad)
            player_records = y[12]
            for name in player_records:
                record = onto.Record(recordcompetitionname=[name], recordgames=[player_records[name][0]],
                                     recordminutes=[player_records[name][1]], recordgoals=[player_records[name][2]],
                                     recordassists=[player_records[name][3]])
                player.playerHasRecord = [record]
                record.recordForSquad = squad

In [6]:

for i in onto.Club.instances():
    print(i)


ontology.Sport_Lisboa_e_Benfica
ontology.Futebol_Clube_do_Porto
ontology.Sporting_Clube_de_Braga
ontology.Sporting_Clube_de_Portugal
ontology.Casa_Pia_Atlético_Clube
ontology.Vitória_Sport_Clube
ontology.Portimonense_Sporting_Clube
ontology.Futebol_Clube_de_Arouca
ontology.Grupo_Desportivo_de_Chaves
ontology.Rio_Ave_Futebol_Clube
ontology.Boavista_Futebol_Clube
ontology.Grupo_Desportivo_Estoril_Praia
ontology.Futebol_Clube_de_Vizela
ontology.Clube_Desportivo_Santa_Clara
ontology.Futebol_Clube_de_Famalicão
ontology.Gil_Vicente_Futebol_Clube
ontology.Club_Sport_Marítimo
ontology.Futebol_Clube_Paços_de_Ferreira
ontology.Moreirense_Futebol_Clube
ontology.Clube_Desportivo_de_Tondela
ontology.B_SAD
ontology.Sporting_Clube_Farense
ontology.Clube_Desportivo_Nacional


In [7]:
onto.save(file="result", format="rdfxml")